# Levelized Cost of Charging Sensitivities  


In [1]:
import sys
import numpy as np
import pandas as pd

sys.path.append('../')
import lcoc.urdb as urdb
import lcoc.afdc as afdc
import lcoc.processing as proc

### LCOC Sensitivity Analysis:

#### 1. Residential TOU:

In [2]:
proc.res_rates_to_utils(scenario = 'no-tou',
                       urdb_rates_file = '../outputs/cost-of-electricity/urdb-res-rates/res_rates.csv',
                       eia_cw_file = '../data/eia/eiaid-crosswalk/eia_crosswalk_17.csv',
                       eia_utils_file = '../data/eia/residential-electricity-prices/eia_residential_17.csv',
                       outpath = '../outputs/cost-of-electricity/res-utilities/')

proc.res_rates_to_utils(scenario="tou-only",
                        urdb_rates_file = '../outputs/cost-of-electricity/urdb-res-rates/res_rates.csv',
                        eia_cw_file = '../data/eia/eiaid-crosswalk/eia_crosswalk_17.csv',
                        eia_utils_file = '../data/eia/residential-electricity-prices/eia_residential_17.csv',
                        outpath = '../outputs/cost-of-electricity/res-utilities/')

Complete, 2008 utilities represented (no TOU rates used).
Complete, 242 utitilies represented (only TOU rates used).


,eiaid,entity,state,customers,eia_cost_per_kwh
0,6641.0,4-County Electric Power Assn,MS,38974,0.1178
1,57394.0,4Change Energy Company,TX,58380,0.0928
2,NaN,A & N Electric Coop,MD,274,0.1149
3,84.0,A & N Electric Coop,VA,31309,0.1112
4,12647.0,"ALLETE, Inc.",MN,122295,0.1057
...,...,...,...,...,...
2003,21114.0,Yazoo Valley Elec Power Assn,MS,9464,0.1409
2004,20997.0,Yellowstone Valley Elec Co-op Inc.,MT,17917,0.1251
2005,NaN,York Electric Coop Inc,NC,13,0.1111
2006,21002.0,York Electric Coop Inc,SC,52788,0.1212


In [3]:
upper_bnd_utils_file = '../outputs/cost-of-electricity/res-utilities/upper_bnd_res_utils.csv'
lower_bnd_utils_file = '../outputs/cost-of-electricity/res-utilities/lower_bnd_res_utils.csv'
upper_bnd_states_file = '../outputs/cost-of-electricity/res-states/res_states_upper_bnd.csv'
lower_bnd_states_file = '../outputs/cost-of-electricity/res-states/res_states_lower_bnd.csv'

proc.res_utils_to_state(utils_file = upper_bnd_utils_file,
                        outfile = upper_bnd_states_file)

proc.res_utils_to_state(utils_file = lower_bnd_utils_file,
                        outfile = lower_bnd_states_file)

Complete, national cost of electricity is $0.13/kWh.
Complete, national cost of electricity is $0.08/kWh.


In [4]:
# VMT assumptions
baseline_vmt = np.array([13065] * 3 + [12582] * 3 + [11432] * 4 + [7812] * 5) #NHTS 2017 (Table 22)
baseline_lifetime_vmt = baseline_vmt.sum()
baseline_aavmt = baseline_lifetime_vmt / 15
low_aavmt = 100000
high_aavmt = 200000

In [5]:
proc.calculate_state_residential_lcoc(coe_file = upper_bnd_states_file,
                                      fixed_costs_path = '../data/fixed-costs/residential/',
                                      aavmt = baseline_aavmt,
                                      outfile = '../outputs/cost-of-charging/residential/res_states_upper_bnd_tou_090820.csv')

proc.calculate_state_residential_lcoc(coe_file = lower_bnd_states_file,
                                      fixed_costs_path = '../data/fixed-costs/residential/',
                                      aavmt = baseline_aavmt,
                                     outfile = '../outputs/cost-of-charging/residential/res_states_lower_bnd_tou.csv')

LCOC calculation complete, national LCOC (residential) is $0.18/kWh
LCOC calculation complete, national LCOC (residential) is $0.12/kWh


In [6]:
proc.calculate_state_residential_lcoc(coe_file = upper_bnd_states_file,
                                      fixed_costs_path = '../data/fixed-costs/residential/',
                                      aavmt = baseline_aavmt,
                                      fraction_home_l1_charging = 0.,
                                      outfile = '../outputs/cost-of-charging/residential/res_states_upper_bnd_090820.csv')

LCOC calculation complete, national LCOC (residential) is $0.18/kWh


In [7]:
proc.combine_res_work_dcfc_lcoc(res_lcoc_file = '../outputs/cost-of-charging/residential/res_states_upper_bnd_tou.csv',
                                outfile = '../outputs/cost-of-charging/comb/comb_states_upper_bnd_tou_090820.csv')

proc.combine_res_work_dcfc_lcoc(res_lcoc_file = '../outputs/cost-of-charging/residential/res_states_lower_bnd_tou.csv',
                                outfile = '../outputs/cost-of-charging/comb/comb_states_lower_bnd_tou_090820.csv')

Combined LCOC calculation complete, national LCOC is $0.17/kWh
Combined LCOC calculation complete, national LCOC is $0.13/kWh


In [8]:
# LCOC for low/baseline/high lifetime VMT scenarios (100k, 162k, & 200k respectively)
df = proc.calculate_state_residential_lcoc(coe_file = '../outputs/cost-of-electricity/res-utilities/res_utils.csv',
                                      fixed_costs_path = '../data/fixed-costs/residential/',
                                      aavmt = low_aavmt,
                                      outfile = '../outputs/cost-of-charging/residential/res_states_upper_bnd_vmt.csv')

proc.calculate_state_residential_lcoc(coe_file = '../outputs/cost-of-electricity/res-utilities/res_utils.csv',
                                      fixed_costs_path = '../data/fixed-costs/residential/',
                                      aavmt = high_aavmt,
                                      outfile = '../outputs/cost-of-charging/residential/res_states_lower_bnd_vmt.csv')

proc.combine_res_work_dcfc_lcoc(res_lcoc_file = '../outputs/cost-of-charging/residential/res_states_upper_bnd_vmt.csv',
                                outfile = '../outputs/cost-of-charging/comb/comb_states_upper_bnd_vmt.csv')

proc.combine_res_work_dcfc_lcoc(res_lcoc_file = '../outputs/cost-of-charging/residential/res_states_lower_bnd_vmt.csv',
                                outfile = '../outputs/cost-of-charging/comb/comb_states_lower_bnd_vmt.csv')

TypeError: cannot convert the series to <class 'float'>

#### 2. Capital Costs - Residential

In [ ]:
proc.calculate_state_residential_lcoc(fraction_home_l1_charging = 0,
                                      aavmt = baseline_aavmt,
                                      outfile = '../outputs/cost-of-charging/residential/res_states_upper_bnd_equip.csv')

proc.calculate_state_residential_lcoc(fraction_home_l1_charging = 1,
                                      aavmt = baseline_aavmt,
                                      outfile = '../outputs/cost-of-charging/residential/res_states_lower_bnd_equip.csv')

In [17]:
proc.calculate_state_residential_lcoc(coe_file = upper_bnd_states_file,
                                      fraction_home_l1_charging = 0,
                                      aavmt = baseline_aavmt,
                                      outfile = '../outputs/cost-of-charging/residential/res_states_upper_bnd_090820.csv')

LCOC calculation complete, national LCOC (residential) is $0.18/kWh


In [7]:
proc.combine_res_work_dcfc_lcoc(res_lcoc_file = '../outputs/cost-of-charging/residential/res_states_upper_bnd_equip.csv',
                                outfile = '../outputs/cost-of-charging/comb/comb_states_upper_bnd_res_equip.csv')

proc.combine_res_work_dcfc_lcoc(res_lcoc_file = '../outputs/cost-of-charging/residential/res_states_lower_bnd_equip.csv',
                                outfile = '../outputs/cost-of-charging/comb/comb_states_lower_bnd_res_equip.csv')

Combined LCOC calculation complete, national LCOC is $0.15/kWh
Combined LCOC calculation complete, national LCOC is $0.11/kWh


#### 3. Capital Costs - Workplace

In [8]:
proc.calculate_state_workplace_public_l2_lcoc(coe_path = '../data/eia/eia_state_commercial_17.csv',
                                              fixed_costs_file = '../data/fixed-costs/com_level2_lower_bnd.txt',
                                              outpath = '../outputs/cost-of-charging/workplace-public-l2/work_pub_l2_states_lower_bnd_equip.csv')

proc.calculate_state_workplace_public_l2_lcoc(coe_path = '../data/eia/eia_state_commercial_17.csv',
                                              fixed_costs_file = '../data/fixed-costs/com_level2_upper_bnd.txt',
                                              outpath = '../outputs/cost-of-charging/workplace-public-l2/work_pub_l2_states_upper_bnd_equip.csv')

LCOC calculation complete, national LCOC (workplace/pub-L2) is $0.12/kWh
LCOC calculation complete, national LCOC (workplace/pub-L2) is $0.25/kWh


In [9]:
proc.combine_res_work_dcfc_lcoc(wrk_lcoc_file = 'outputs/cost-of-charging/workplace-public-l2/work_pub_l2_states_lower_bnd_equip.csv',
                                outfile = 'outputs/cost-of-charging/comb/comb_states_lower_bnd_work_equip.csv')

proc.combine_res_work_dcfc_lcoc(wrk_lcoc_file = 'outputs/cost-of-charging/workplace-public-l2/work_pub_l2_states_upper_bnd_equip.csv',
                                outfile = 'outputs/cost-of-charging/comb/comb_states_upper_bnd_work_equip.csv')

Combined LCOC calculation complete, national LCOC is $0.14/kWh
Combined LCOC calculation complete, national LCOC is $0.16/kWh


#### 4. Capital Costs - DCFC

In [10]:
proc.combine_dcfc_profiles_into_single_lcoc(dcfc_lcoc_file = 'outputs/cost-of-charging/dcfc/dcfc_states_lower_bnd_equip.csv',
                                           load_profile_path = 'data/dcfc-load-profiles/')

proc.combine_dcfc_profiles_into_single_lcoc(dcfc_lcoc_file = 'outputs/cost-of-charging/dcfc/dcfc_states_upper_bnd_equip.csv',
                                           load_profile_path = 'data/dcfc-load-profiles/')

LCOC calculation complete, national LCOC (DCFC) is $0.18/kWh
LCOC calculation complete, national LCOC (DCFC) is $0.27/kWh


In [11]:
proc.combine_res_work_dcfc_lcoc(dcfc_lcoc_file = 'outputs/cost-of-charging/dcfc/dcfc_states_lower_bnd_equip.csv',
                                outfile = 'outputs/cost-of-charging/comb/comb_states_lower_bnd_dcfc_equip.csv')

proc.combine_res_work_dcfc_lcoc(dcfc_lcoc_file = 'outputs/cost-of-charging/dcfc/dcfc_states_upper_bnd_equip.csv',
                                outfile = 'outputs/cost-of-charging/comb/comb_states_upper_bnd_dcfc_equip.csv')

Combined LCOC calculation complete, national LCOC is $0.14/kWh
Combined LCOC calculation complete, national LCOC is $0.15/kWh


#### 5. Combined (best case/worst case scenario)  
**Combined (best case)**:  
* Charging Mix: 100% Res  
* Residential TOU: TOU rates
* Capital Costs (residential): None, L1 only  

**Combined (worst case)**:
* Charging Mix: 100% DCFC  
* Capital Costs (DCFC): 95th percentile estimates + 500k (p3), 2.5M (p4) grid enhancements

In [12]:
proc.calculate_state_residential_lcoc(coe_file = 'outputs/cost-of-electricity/res-states/res_states_lower_bnd.csv',
                                      fraction_home_l1_charging = 1,
                                      outfile = 'outputs/cost-of-charging/residential/res_states_lower_bnd_equip_tou.csv')

LCOC calculation complete, national LCOC (residential) is $0.08/kWh


In [13]:
proc.combine_dcfc_profiles_into_single_lcoc(dcfc_lcoc_file = 'outputs/cost-of-charging/dcfc/dcfc_states_upper_bnd_equip.csv',
                                           load_profile_path = 'data/dcfc-load-profiles/')

LCOC calculation complete, national LCOC (DCFC) is $0.27/kWh
